In [1]:
# import prereq.
from __future__ import absolute_import, division, print_function, unicode_literals
 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping # used for callback 


import pandas as pd 
from sklearn import preprocessing
import numpy as np

In [2]:
# helper: normalize

def normalize_df(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(x_scaled)

In [3]:
# read the datasets
train = pd.read_csv("train.csv") 
cross = pd.read_csv("cross.csv") 
test = pd.read_csv("test.csv") 

In [4]:

# split in X & Y

# training set
y = train.iloc[:, -2:]
x = normalize_df(train.iloc[:, 0:-2])
x = x.drop(x.columns[0], axis=1)


# dev set

y_c = cross.iloc[:, -2:]
x_c = normalize_df(cross.iloc[:, 0:-2])
x_c = x_c.drop(x_c.columns[0], axis=1)


# test set

y_t = test.iloc[:, -2:]
x_t = normalize_df(test.iloc[:, 0:-2])
x_t = x_t.drop(x_t.columns[0], axis=1)

In [5]:
# convert to numpy

x = np.asarray(x.to_numpy())
y = np.asarray(y.to_numpy())
x_t = np.asarray(x_t.to_numpy())
y_t = np.asarray(y_t.to_numpy())
x_c = np.asarray(x_c.to_numpy())
y_c = np.asarray(y_c.to_numpy())

In [6]:
# arhitect the model

model = tf.keras.Sequential([
layers.Dense(64, activation='relu', input_shape=(32,)),
layers.Dropout(0.15),
layers.Dense(128, activation='relu'),
layers.Dropout(0.25),
layers.Dense(128, activation='relu'),
layers.Dropout(0.25),
layers.Dense(128, activation='relu'),
layers.Dropout(0.25),
layers.Dense(64, activation='relu'),
layers.Dropout(0.25),
layers.Dense(64, activation='relu'),
layers.Dropout(0.15),
layers.Dense(64, activation='relu'),
layers.Dropout(0.15),
layers.Dense(64, activation='relu'),
layers.Dropout(0.15),
layers.Dense(32, activation='relu'),
layers.Dropout(0.1),
layers.Dense(32, activation='relu'),
layers.Dropout(0.1),
layers.Dense(32, activation='relu'),
layers.Dropout(0.1),
layers.Dense(32, activation='relu'),
layers.Dropout(0.1),
layers.Dense(16, activation='relu'),
layers.Dropout(0.05),
layers.Dense(16, activation='relu'),
layers.Dropout(0.05),
layers.Dense(2, activation='softmax')])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#the training process stops because of the val_acc – min_delta < baseline for the patience interval (10 epochs). 
callbacks = [EarlyStopping(monitor='val_acc',verbose=1,min_delta=0.00000001, patience=10)]

In [8]:
model.fit(x.astype('float32'),y, 
          epochs=50, 
          batch_size=99, 
          shuffle=True, 
          validation_data=(x_c.astype('float32'), y_c),
          callbacks=callbacks)


Train on 99000 samples, validate on 500 samples
Epoch 1/50
99000/99000 [==============================] - 16s 164us/sample - loss: 0.2259 - acc: 0.8997 - val_loss: 5.1615e-05 - val_acc: 1.0000
Epoch 2/50
99000/99000 [==============================] - 14s 142us/sample - loss: 0.0040 - acc: 0.9990 - val_loss: 7.7194e-06 - val_acc: 1.0000
Epoch 3/50
99000/99000 [==============================] - 14s 143us/sample - loss: 0.0021 - acc: 0.9996 - val_loss: 2.0859e-06 - val_acc: 1.0000
Epoch 4/50
99000/99000 [==============================] - 14s 145us/sample - loss: 0.0025 - acc: 0.9994 - val_loss: 2.1635e-05 - val_acc: 1.0000
Epoch 5/50
99000/99000 [==============================] - 14s 145us/sample - loss: 0.0015 - acc: 0.9996 - val_loss: 3.8358e-06 - val_acc: 1.0000
Epoch 6/50
99000/99000 [==============================] - 15s 147us/sample - loss: 0.0014 - acc: 0.9996 - val_loss: 7.6190e-07 - val_acc: 1.0000
Epoch 7/50
99000/99000 [==============================] - 15s 151us/sample - loss:

In [11]:
model.evaluate(x_t, y_t) 

500/500 [==============================] - 0s 236us/sample - loss: 2.2173e-08 - acc: 1.0000


[2.2172822198740504e-08, 1.0]

In [12]:
# with this config i achived acc on trian set(99k values) of 99.99%
# and acc 100% on test and dev seds
model.save('9997predict.h5')

In [ ]:
# auth: Martinescu Sorin-Alexandru 